In [1]:
%load_ext autoreload
%autoreload 2

In [64]:
from __init__ import *
import numpy as np
from export.saver import read_pickle
from helpers.layout import Layout
from helpers.graph_helpers import *
from helpers.helpers import sort_and_group_objects
from typing import Dict
from domains.domain import Domain
from helpers.directions import get_axis, get_opposite_axis, Direction
from placement2.arrange import *
from copy import copy, deepcopy
from shapely import LineString, Point, Polygon, BufferCapStyle, BufferJoinStyle, buffer

In [72]:
layout: Layout = read_pickle("1001_amber_c_ag")
G = layout.graph

In [73]:
arr, *_ = initialize_arr(layout)
arr

array([['m_bed', 'bed1'],
       ['m_closet', ''],
       ['m_bath', ''],
       ['bath', ''],
       ['den_closet', ''],
       ['kitchen', ''],
       ['living', '']], dtype=object)

In [74]:
arr1 = place_next_row(G, arr)
arr1

ic| north_row: ['m_bed', 'bed1']
ic| north_east_members: []


['bed1_closet', 'transit', 'm_closet']


array([['m_bed', 'bed1', ''],
       ['m_closet', 'transit', 'bed1_closet'],
       ['m_bath', '', ''],
       ['bath', '', ''],
       ['den_closet', '', ''],
       ['kitchen', '', ''],
       ['living', '', '']], dtype=object)

In [75]:
arr2 = place_next_row(G, arr1)
arr2

ic| north_row: ['m_closet', 'transit', 'bed1_closet']
ic| north_east_members: []


['laundry', 'm_bath', 'linen', 'corridor']


array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', '', '', ''],
       ['den_closet', '', '', ''],
       ['kitchen', '', '', ''],
       ['living', '', '', '']], dtype=object)

In [38]:
def get_north_and_east_nbs(G, arr, ix, node):
    north_row = get_row(arr, ix)
    return [i for i in north_row if i in G.nodes()[node]["data"]["EAST"]]



In [40]:
get_north_and_east_nbs(G, arr2, 2, "den")

[]

In [53]:
G.nodes["m_bath"]

{'data': NeighborDirections(NORTH=['transit', 'm_closet'], SOUTH=['bath'], EAST=['transit', 'corridor'], WEST=[])}

In [76]:
arr3 = place_next_row(G, arr2)
arr3

ic| north_row: ['m_bath', 'corridor', 'linen', 'laundry']
ic| north_east_members: [('bath', 'corridor')]


['den', 'dining', 'bath']


array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', '', '', ''],
       ['den_closet', '', '', ''],
       ['kitchen', '', '', ''],
       ['living', '', '', '']], dtype=object)

In [16]:
find_east_nb(G, "bath", get_row(arr3, 2))

'corridor'

In [18]:
find_east_nb(G, "corridor", ['den', 'dining', 'bath'])

'den'

In [7]:
def create_row(G, arr, ix):
    valid_nodes = get_next_row_all(ix, G, arr)
    found_nodes = [arr[ix+1, 0]]
    avail_nodes = list(set(valid_nodes).difference(set(found_nodes)))
    print(avail_nodes, found_nodes)

    max_iter = 10
    cnt = 0

    while avail_nodes:
        cnt+=1
        curr_node = found_nodes[-1]
        next_node = find_east_nb(G, curr_node, avail_nodes)
        if not next_node:
            return found_nodes
        else:
            found_nodes.append(next_node)
            curr_node = next_node
        if cnt > max_iter:
            raise Exception("Exceeded max iter")


In [10]:
create_row(layout.graph, arr, 0)

['transit', 'bed1_closet'] ['m_closet']
m_closet
transit
bed1_closet


['m_closet', 'transit', 'bed1_closet']

In [63]:
from itertools import count

In [67]:
count()

count(0)

In [35]:
{1,2,3}.difference({1})

{2, 3}

In [61]:
create_row(layout.graph, arr, 0)

{'transit', 'bed1_closet'} ['m_closet']


In [25]:
def find_east_nb(G, node, north_nbs):
    valid_nbs = [i for i in north_nbs if i != node]
    east_nbs = G.nodes()[node]["data"]["EAST"]
    res = set(east_nbs).intersection(set(valid_nbs))
    if len(res) == 1:
        return res
    if len(res) == 0:
        return None
    if len(res) > 1:
        raise Exception("More than one east nb")

In [62]:
find_east_nb(layout.graph, w_node, valid_nodes)

0
1
2
3


array([['m_bed', 'bed1', '', ''],
       ['m_closet', 'transit', 'bed1_closet', ''],
       ['m_bath', 'corridor', 'linen', 'laundry'],
       ['bath', 'dining', 'den', ''],
       ['kitchen', 'living', 'den_closet', ''],
       ['', '', '', '']], dtype=object)